# anomaly → PMML (using Nyoka)

### Exporter: Anomaly Detection models (OneClassSVM)
### Data Set used: iris 

### **STEPS**: 
- Build the model using sklearn OneClassSVM
- Build PMML (Data Dictionary, Mining schema, Ouput, PMML) using Nyoka classes

# Pre-processing, Model building (using pipeline) for iris data set

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from requests.auth import HTTPBasicAuth
import requests
import json
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Imputer
from sklearn.svm import OneClassSVM

irisdata = datasets.load_iris()
iris = pd.DataFrame(irisdata.data,columns=irisdata.feature_names)
iris['Species'] = irisdata.target

feature_names = iris.columns.drop('Species')

X_train, X_test, y_train, y_test = train_test_split(iris[iris.columns.drop(['Species'])], 
                                                    iris['Species'], test_size=0.33, random_state=101)

# X_test.to_csv("iris_test.csv")


# pipe = Pipeline([('standard_scaler',StandardScaler()), ('Imputer',Imputer()), ('model',OneClassSVM())])

pipe = Pipeline([('standard_scaler',StandardScaler()), ('Imputer',Imputer())])

X_train = pipe.fit_transform(X_train)

ocs = OneClassSVM()

ocs.fit(X_train)

print("\n","Anomaly detection model is built successfully.")


 Anomaly detection model is built successfully.


C:\Users\cchi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


# Export the Pipeline object into PMML using the Nyoka package

In [2]:
from nyoka import model_to_pmml
pmml_file_name = "OneClassSVM_model.pmml"

In [3]:
toExportDict={
    'model1':{
        'hyperparameters':None,
        'preProcessingScript':None,
        'pipelineObj':pipe,
        'modelObj':ocs,
        'featuresUsed':feature_names,
        'targetName':'Species',
        'postProcessingScript':None,
        'taskType': 'trainAndscore'
    }
}
pmml = model_to_pmml(toExportDict, pmml_f_name=pmml_file_name)